In [1]:
import sys
import os
import queue
import sounddevice as sd
import vosk
import json

In [2]:

# Define your model path here
MODEL_PATH = "../vosk-model-small-en-us-0.15"

# Define keywords you want to detect
KEYWORDS = ["hello", "world", "python", "game"]

# Set up Vosk model and recognizer
if not os.path.exists(MODEL_PATH):
    print(f"Model path {MODEL_PATH} does not exist. Please check the path.")
    sys.exit(1)

model = vosk.Model(MODEL_PATH)
recognizer = vosk.KaldiRecognizer(model, 16000)
recognizer.SetWords(True)  # Enable word detection

# Set up audio stream
q = queue.Queue()

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from ../vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from ../vosk-model-small-en-us-0.15/graph/HCLr.fst ../vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo ../vosk-model-small-en-us-0.15/graph/phones/word_boundary.int


In [3]:
def callback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    q.put(bytes(indata))

poem = "Once upon a midnight dreary, while I pondered, weak and weary, Over many a quaint and curious volume of forgotten lore— While I nodded, nearly napping, suddenly there came a tapping, As of some one gently rapping, rapping at my chamber door. “’Tis some visitor,” I muttered, “tapping at my chamber door— Only this and nothing more.”"
poem = "is there balm in gilead, tell me, tell me, i implore!"
poem = "one two three"
stripped_poem = ''.join(e for e in poem if e.isalnum() or e.isspace())
poem_words = stripped_poem.split()
poem_words = [word.lower() for word in poem_words]
poem_words += ["[unk]"]  # Add [unk] token for unknown words
grammar_str = json.dumps(poem_words)
recognizer.SetGrammar(grammar_str)


try:
    with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16',
                           channels=1, callback=callback):
        print("Start speaking...")

        while True:
            data = q.get()
            if recognizer.AcceptWaveform(data):
                result = recognizer.Result()
                result_dict = json.loads(result)
                heard_words = result_dict.get("result", [])
                for w in heard_words:
                    print(w['word'], w['start'], w['end'])
            else:
                partial_result = recognizer.PartialResult()
                partial_dict = json.loads(partial_result)
                print("Partial result:", partial_dict.get("partial"))

except KeyboardInterrupt:
    print("\nDone")
except Exception as e:
    print(f"An error occurred: {str(e)}")

LOG (VoskAPI:UpdateGrammarFst():recognizer.cc:287) ["one", "two", "three", "[unk]"]
LOG (VoskAPI:Estimate():language_model.cc:142) Estimating language model with ngram-order=2, discount=0.5
LOG (VoskAPI:OutputToFst():language_model.cc:209) Created language model with 5 states and 8 arcs.


Start speaking...
Partial result: 
Partial result: 
Partial result: 
Partial result: one
Partial result: one two three
Partial result: one two three
one 1.62 1.89
two 1.89 2.1
three 2.1 2.49
Partial result: one
Partial result: one
Partial result: one

Done


In [ ]:
# use vosk and a known grammar to perform forced alignment

grammar = "testing one two three"
